# Critical Beta for 3D Quantum Spin Systems

In [4]:
import math
import numpy as np
import bisect

In [4]:
            ################# How to set up a calculation ################# 
# 1. Set the parameters: Beta, Lenght(L), the rate of crosses(u), theta(=2,3), Edges 
# 2. Create the intial configuration using RandomInitialConfiguration(beta,u,L) and assort the
#    the output lists using GetBundle. The output, Lists, be indexed by the vertex number and
#    contains a tuple of lists that contain Link times, the vertex it connects too, and the type
#    of link. 
# 3. (Optional) Traverse the random configuration to see the 'Shadow Lengths' and Loops.
# 4. Run the MCMC code on the lists for desired number of itertions 

In [ ]:
                ################# Table of Contents ################# 
    
# 'Markov Chain Monte Carlo' is the end product, it's the actual Monte Carlo simulation that interacts with a
# a global list.

# 'The set up' is just a block of code that generates a lattice with respect to its edges and adherence to periodic
# boundary conditions.

# 'Initial Configuration & arranging all the data' creates a random intial interchange model using np.poisson mostly
# and the GetBundle function arranges the configuration into a list indexed by the lattice vertex. Each entry
# is compromised of three lists e.g. for the zero-th lattice site (origin)...
# Lists[0]=[[times of links],[neighbour it connects to], [type of link]]

# 'Machinery to traverse loops and post-mortem analysis' is just a collection of code that I use to traverse the loops
# and get the information needed. These codes interact with the lists produced in the previous section.

# 'Auxiliary functions to facilitate MCMC' essentially all the code here is a 'special case' of the code in the 
# previous blocks, but act more locally and allow changes to be proposed and acted on 

## Markov Chain Monte Carlo

In [22]:
#takes in the lists from the RandomInitialConfiguration once its been arranged to a list by the vertices
#also takes in the list of egdes that make up the lattice
#this is a single 'sweep' at the moment

def MCMC(Edges,beta,L,u,theta):
    for i in range(3*(L**3)):
        x,y=Edges[i]
        
        #consolidates all the links over a given edge
        connected_vertices=np.array(Lists[x][1]) 
        xy_links,=(np.where((connected_vertices)==y))  
        link_times_on_xy=np.array(Lists[x][0])[xy_links]
        link_types_on_xy=np.array(Lists[x][2])[xy_links]
        w_e=len(link_times_on_xy) #no. of links on edge in question
        
        #creates the 'c.p.d'
        a=u*beta/((beta+1)*(w_e+1))
        b=(1-u)*beta/((beta+1)*(w_e+1))
        c=1/(beta+1)
        
        rand=np.random.rand()
        
        # the chances of adding a cross on the edge
        if 0 <= rand < a:
            
            if w_e==0:
            
                link_time=np.random.rand()*beta
                metro_step=insertion_prop(x,y,link_time,1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,1)
                    print 'a',i
                    
            elif w_e==1:
                
                #so the cross will either be first or second 
                if np.random.randint(0,2)==0: #first
                    link_time=link_times_on_xy[0]/2.0
                    
                else: #second 
                    link_time=(beta-link_times_on_xy[0])/2.0
                    
                metro_step=insertion_prop(x,y,link_time,1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,1)
                    print 'b',i
            
            else:
                
                nth_pos=np.random.randint(0,w_e+1)
                
                if nth_pos==0: #lowest link
                    link_time=link_times_on_xy[0]/2.0
                    
                elif nth_pos==w_e: #highest link
                    link_time=(beta+link_times_on_xy[-1])/2.0
                
                else: #some intermeddiate link
                    link_time=(link_times_on_xy[nth_pos]+link_times_on_xy[nth_pos-1])/2.0
              
                metro_step=insertion_prop(x,y,link_time,1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,1)
                    print 'c',i
            
                
        # the chances of adding a bar on the said edge
        elif a <= rand < a+b:
            
            if w_e==0:
                
                link_time=np.random.rand()*beta
                metro_step=insertion_prop(x,y,link_time,-1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,-1)
                    print 'd',i
                    
            elif w_e==1:
                
                #so the cross will either be first or second 
                if np.random.randint(0,2)==0: #first
                    link_time=link_times_on_xy[0]/2.0
                        
                else: #second 
                    link_time=(beta-link_times_on_xy[0])/2.0
                    
                metro_step=insertion_prop(x,y,link_time,-1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,-1)
                    print 'e',i
            
            else:
                
                nth_pos=np.random.randint(0,w_e+1)
                
                if nth_pos==0: #lowest link
                    link_time=link_times_on_xy[0]/2.0
                    #print 'a',e
                    
                elif nth_pos==w_e: #highest link
                    link_time=(beta+link_times_on_xy[-1])/2.0
                    #print 'b',e
                
                else: #some intermeddiate link
                    link_time=(link_times_on_xy[nth_pos]+link_times_on_xy[nth_pos-1])/2.0
                    #print 'c',e
              
                metro_step=insertion_prop(x,y,link_time,-1)
                
                if metro_step > np.random.rand():
                    insert_link(x,y,link_time,-1)
                    print 'f',i
            
        # the chances of deleting a link
        elif a+b <= rand < a+b+c:
            if w_e==0:
                print 'g',i
                continue
            
            else:
                
                #randomly chosen a link to delete 
                L=np.random.randint(0,w_e)
                link_time=link_times_on_xy[L]
                link_type=link_types_on_xy[L]
                
                #need to work out the effects of deleting the link on L 
                metro_step=deletion_prop(x,y,link_time,link_type)
                
                if metro_step > np.random.rand():
                    delete_link(x,y,link_time)
                    print 'h',i
                    
        # doing nothing
        else:
            print 'i',i
            continue 

## The set up

In [8]:
#just a collection of code to set up the set edges
#creates a list of all the edges with periodic boundary conditions
#standard, Peter's code works for any d so that makes life easy

def areAdjacentInZd(x,y,L,d):  
    if x==y:
        return False
    dist1Coords = 0   
    for i in range(d):
        ithDiff = abs(math.floor(x/L**i)%L-math.floor(y/L**i)%L)
        if ithDiff == 0:
            continue
        if ithDiff in [1,L-1]: 
            dist1Coords+=1
            if dist1Coords>1: # if two or more coordinates are distance 1 apart, they are not neighbours
                return False
        else: # if one coordinate is more than distance 1 apart, they are not neighbours
            return False
    return True


#generates a lattice with regard to edegs 
def generateZd(L,d):
    edges = []
    for x in range(L**d):
        for y in range(x,L**d): # don't want [0,1] _and_ [1,0] as an edge
            if areAdjacentInZd(x,y,L,d):
                edges.append([x,y])   
    #return edges, len(edges), L**d
    return edges

## Initial Configuration & arranging all the data

In [9]:
#generates the number of links we are expected to have and their types depending on u, 
#creates a randomised list crosses and double bars, scatter these N crosses/bars on [0,beta]
#E is the number of edges

def RandomInitialConfiguration(beta,u,L):
    E=generateZd(L,3)
    # each link is either a cross or a bar with probability u or 1-u, respectively
    N = np.random.poisson(len(E)*beta)
    vertex=np.zeros(N,  dtype=np.int)
    adj_vertex=np.zeros(N, dtype=np.int)
    link_pos = np.random.rand(N)*beta
    crossOrBar = np.random.choice([1, -1], N, p=[u, 1-u])
    edges=[]
    randomize_edges=np.random.randint(0,len(E),N)
    for i in randomize_edges:
        edges.append(E[i])
    for i in range(N):
        x,y=edges[i]
        vertex[i]=x
        adj_vertex[i]=y
    return link_pos,crossOrBar,vertex,adj_vertex

In [10]:
#generates a massive list, index of the list is a labelled lattice site
#the output is a list of tuples with the main index as the vertex, the three lists are link positions, vertices 
#they are connected to & the type of links

def GetBundle(L,link_pos,crossOrBar,vertex,adj_vertex):
    Lists=[]
    for x in range(L**3):
        l,=np.where(np.array(vertex)==x)
        k,=np.where(np.array(adj_vertex)==x)
        
        #we check both for the the site because we want to include all the links, not just one-way
        link_times=[]
        vertices=[]
        links=[]
        
        for i in l:
            link_times.append(link_pos[i])
            vertices.append(adj_vertex[i])
            links.append(crossOrBar[i])
            
        for i in k:
            link_times.append(link_pos[i])
            vertices.append(vertex[i])
            links.append(crossOrBar[i])
            
        vertices=[x for _,x in sorted(zip(link_times,vertices))]
        links=[x for _,x in sorted(zip(link_times,links))]
        link_times.sort()
        Lists.append((link_times,vertices,links))
    return Lists

## Machinery to traverse loops and post-mortem analysis

In [11]:
#this is used in traverse for the entire configuration

def next_move(x,s,di):
    link_times=Lists[x][0]
    vertices=Lists[x][1]
    links=Lists[x][2]
    s_prev=s
    
    if di==1:
        if s==0: 
            s=link_times[0] #will move to the first link
            x=vertices[0] #moves to the vertex joint by the link
            l=links[0] #stores the type of link for next move
        
        elif len(link_times)-1==np.where(np.array(link_times)==s)[0][0]:
        #elif len(link_times)-1==link_times.index(s):
            #s-->0 if there is no more links above the current position and di=1 then goes back to 0 due P.B.C.
            s=0 
            l=1
            
        else:
            s=link_times[np.where(np.array(link_times)==s_prev)[0][0]+1] 
            x=vertices[np.where(np.array(link_times)==s_prev)[0][0]+1]
            l=links[np.where(np.array(link_times)==s_prev)[0][0]+1]
    
    elif di==-1:
        if s==0: 
            s=link_times[len(link_times)-1] #will move to the first link "below" current vertex by p.b.c
            x=vertices[len(link_times)-1] #moves to the vertex joint by the link
            l=links[len(link_times)-1] #stores the type of link for next move
            
        elif np.where(np.array(link_times)==s)[0][0]==0:#this is to check if the we're at the last link on that vertex
        #elif link_times.index(s)==0:
            #due to P.b.c its sent to the t=0 as it goes thru a vertex
            s=0
            l=1
            
        else:
            s=link_times[np.where(np.array(link_times)==s_prev)[0][0]-1] 
            x=vertices[np.where(np.array(link_times)==s_prev)[0][0]-1]
            l=links[np.where(np.array(link_times)==s_prev)[0][0]-1]
        
    return x,s,l #vertex, time and cross/bar

#so this code works nicely if we're interested in figuring out the shadow length and use this in traverse

In [12]:
# all used for the entire configuration

def traverse():
    VisitedVertex=[]
    Paths=[]
    Lengths=[]
    for i in range(L**3):
        di=1
        Loop=[]
        diretions=[]
        path=[]
        x=i
        x_init=x
        #to get this to work we start at s=0 every time
        s=0  
        #by default we start by moving up
        l_init=1      
        Loop.append(x)
        if len(Lists[i][0])==0:
            Lengths.append(np.array([x]))
            continue
        
        elif i in VisitedVertex:
            continue
            
        while True: #need to create a more effecient way for this loop...
                x,s,l=next_move(x,s,di)
                if s==0:
                    if x==x_init:
                        break
                    Loop.append(x)
                    VisitedVertex.append(x)
                    diretions.append(di)
                di=di*l
                path.append((x,s,l))
        Lengths.append(np.array(Loop))
        Paths.append(path)

    np.array(Lengths)                    
    #return AllLoops,approach,Paths
    #return Lengths
    return Lengths

#this chunky looking code works effectively to figure out the loops present, but only records the loops with
#a shadow length greater or equal to one

In [13]:
def second_moments():
    Loops=traverse()
    tot=0
    for i in Loops:
        tot+=(len(i)/float(L**3))**2
    return tot

## Auxiliary functions to facilitate MCMC

In [14]:
def traverse_4_deletion(x,y,s,Lists):
    path=[]
    x_init=x
    s_init=s

    c=1 #we start off by moving along the heights
    path.append((x_init,s_init,1))
    #since starting off from any arbitrary position we'll move up, we look for the next biggest spacetime pnt
    
    #this moves to the next space-time point where theres a link
    di=1
    while True:
        x,s,l,c=next_move_v2(x,s,c,di,Lists)
        #di=di*l
        
        path.append((x,s,di,c))
        
        if s==s_init:
            if x==x_init:
                break
            elif x==y:
                break 
                
        di=di*l
        

    #return path,x,di
    return x,di

#now to use the traverse_4_deletion to delete a link in question and tell us del_L
def deletion_prop(x,y,s,l):
    
    approach_vertex,approach_direction=traverse_4_deletion(x,y,s,Lists)
    
    #now the independent cases for crosses
    if l==1:
        if approach_direction==1:
            if approach_vertex==x:
                return 1
            elif approach_vertex==y:
                #return -1
                return 1/2.0
        else:
            #return 0
            return 1/np.sqrt(2)
        
    if l==-1:
        if approach_direction==1:
            if approach_vertex==x:
                return 1
            elif approach_vertex==y:
                return 1/np.sqrt(2)
        else:
            return 1/2.0  

In [15]:
def traverse_4_insertion(x,y,s,Lists):
    
    bisect.insort(Lists[x][0],s)
    bisect.insort(Lists[y][0],s)
    
    index_x=Lists[x][0].index(s)
    index_y=Lists[y][0].index(s)
    
    Lists[x][1].insert(index_x,y)
    Lists[y][1].insert(index_y,x)
    
    Lists[x][2].insert(index_x,2)
    Lists[y][2].insert(index_y,2)
    
    path=[]
    x_init=x
    s_init=s
    #l_init=l
    c=1 #we start off by moving along the heights
    path.append((x_init,s_init,1))
    #since starting off from any arbitrary position we'll move up, we look for the next biggest spacetime pnt
    
    #this moves to the next space-time point where theres a link
    di=1
    while True:
        x,s,l,c=next_move_v2(x,s,c,di,Lists)
        path.append((x,s,di,c))
    
        if l==2:
            break
        di=di*l
        
    #delete_link(x,y,s)
    
    del Lists[x_init][0][index_x]
    del Lists[x_init][1][index_x]
    del Lists[x_init][2][index_x]
    
    del Lists[y][0][index_y]
    del Lists[y][1][index_y]
    del Lists[y][2][index_y]
    
    return path
    #return x,di

#now to use the traverse_4_insertion to insert a link in question and tell us del_L

def insertion_prop(x,y,s,l):
    
    approach_vertex,approach_direction=traverse_4_insertion(x,y,s,Lists)
    
    #prob for adding crosses
    if l==1:
        if approach_direction==1:
            if approach_vertex==x:
                return 1/2.0
            elif approach_vertex==y:
                #return -1
                return 1
        else:
            #return 0
            return 1/np.sqrt(2)
        
    if l==-1:
        if approach_direction==1:
            if approach_vertex==x:
                return 1/2.0
            elif approach_vertex==y:
                return 1/np.sqrt(2)
        else:
            return 1

In [16]:
#this works for when we're considering a link to be removed
#this considers a horizontal movement as an extra step,doesnt seem significanlty slower atm 

def next_move_v2(x,s,c,di,AllPositions):
    link_times=AllPositions[x][0]
    vertices=AllPositions[x][1]
    links=AllPositions[x][2]
    s_prev=s
    
    #an additional criterion for when c=0 means a link is being traversed
    if c==0:
        x=vertices[np.where(np.array(link_times)==s_prev)[0][0]]
        l=1
        c=1
        
        
    elif c==1:

        if di==1:
            if len(link_times)-1==link_times.index(s):
                s=link_times[0] #will move to the first link
                #x=vertices[0] #moves to the vertex joint by the link
                l=links[0] #stores the type of link for next move
                c=0
            
            else:
                s=link_times[np.where(np.array(link_times)==s_prev)[0][0]+1] 
                #x=vertices[np.where(np.array(link_times)==s_prev)[0][0]+1]
                l=links[np.where(np.array(link_times)==s_prev)[0][0]+1]
                c=0
    
        elif di==-1:
            if link_times.index(s)==0: 
                s=link_times[len(link_times)-1] #will move to the first link "below" current vertex by p.b.c
                #x=vertices[len(link_times)-1] #moves to the vertex joint by the link
                l=links[len(link_times)-1] #stores the type of link for next move
                c=0
        
            else:
                s=link_times[np.where(np.array(link_times)==s_prev)[0][0]-1] 
                #x=vertices[np.where(np.array(link_times)==s_prev)[0][0]-1]
                l=links[np.where(np.array(link_times)==s_prev)[0][0]-1]
                c=0
        
    return x,s,l,c #vertex, time and cross/bar

In [17]:
def delete_link(x,y,s):
    
    #these two finds the index of link, its position
    x_index=Lists[x][0].index(s)
    y_index=Lists[y][0].index(s)
    
    #deletes the link off both vertices 
    del Lists[x][0][x_index]
    del Lists[x][1][x_index]
    del Lists[x][2][x_index]
    
    del Lists[y][0][y_index]
    del Lists[y][1][y_index]
    del Lists[y][2][y_index]
    
def insert_link(x,y,s,l):
    
    #adding time first 
    bisect.insort(Lists[x][0],s)
    bisect.insort(Lists[y][0],s)
    
    index_x=Lists[x][0].index(s)
    index_y=Lists[y][0].index(s)
    
    #add the neighbouring vertex now 
    Lists[x][1].insert(index_x,y)
    Lists[y][1].insert(index_y,x)
    
    #... and the link type
    Lists[x][2].insert(index_x,l)
    Lists[y][2].insert(index_y,l)

In [18]:
def temp_add_link(x,y,s,AllPositions):
    
    LinkTimes_x=AllPositions[x][0][:]
    LinkVert_x=AllPositions[x][1][:]
    LinkTypes_x=AllPositions[x][2][:]
    
    OrigXTuple=(LinkTimes_x[:],LinkVert_x[:],LinkTypes_x[:])
    
    LinkTimes_y=AllPositions[y][0][:]
    LinkVert_y=AllPositions[y][1][:]
    LinkTypes_y=AllPositions[y][2][:]
    
    OrigYTuple=(LinkTimes_y[:],LinkVert_y[:],LinkTypes_y[:])
    
    bisect.insort(LinkTimes_x,s)
    bisect.insort(LinkTimes_y,s)
    
    LinkVert_x.insert(LinkTimes_x.index(s),y)
    LinkVert_y.insert(LinkTimes_y.index(s),x)
    
    LinkTypes_x.insert(LinkTimes_x.index(s),2)
    LinkTypes_y.insert(LinkTimes_y.index(s),2)
    
    TempXTuple=(LinkTimes_x,LinkVert_x,LinkTypes_x)
    TempYTuple=(LinkTimes_y,LinkVert_y,LinkTypes_y)
    
    return TempYTuple,TempXTuple,OrigYTuple,OrigXTuple


def traverse_4_insertion(x,y,s,AllPositions):
        
    TempYTuple,TempXTuple,OrigYTuple,OrigXTuple=temp_add_link(x,y,s,AllPositions)
    
    AllPositions[x]=TempXTuple
    AllPositions[y]=TempYTuple
    
    
    path=[]
    x_init=x
    s_init=s
    #l_init=l
    c=1 #we start off by moving along the heights
    path.append((x_init,s_init,1))
    #since starting off from any arbitrary position we'll move up, we look for the next biggest spacetime pnt
    
    #this moves to the next space-time point where theres a link
    di=1
    while True:
        x,s,l,c=next_move_v2(x,s,c,di,AllPositions)
        path.append((x,s,di,c))
        if l==2:
            break
        di=di*l

    AllPositions[x_init]=OrigXTuple
    AllPositions[y]=OrigYTuple

    #return path
    return x,di